## Inprocessing - Meta Fair Classifier  -  Compas Model
- for 'sex' and 'race'

In [5]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.insert(0, project_root)

import pandas as pd
from src.data_loading import load_compas_sex, load_compas_race
from src.modeling import meta_fair_classifier_train_and_predict
from src.metrics import compute_metrics, compare_viz_metrics_2x3, save_agg_metrics, save_raw_metrics
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd

In [6]:
dataset_name = 'compas'
mitigation_name   = 'meta fair classifier'
pipeline_stage    = 'inprocessing'  
out_dir_plots    = '../../reports/plots_compas'

In [7]:
baseline_race_agg = pd.read_csv('../../reports/baseline_agg/compas_race_metrics_agg.csv', index_col=0)
baseline_sex_agg = pd.read_csv('../../reports/baseline_agg/compas_sex_metrics_agg.csv', index_col=0)

--------------------------------------------------------------------------------------------------------------------------------------

## default MFC, race

In [8]:
# 1) Retrieve data
protected           = 'race'
privileged_value    = 0.0
unprivileged_value  = 1.0

ds, df = load_compas_race()  
favorable = ds.favorable_label      # COMPAS: 0.0
unfavorable = ds.unfavorable_label  # COMPAS: 1.0
feature_cols = [c for c in df.columns if c not in ('label','race')]

# 2) Run experiment, Evaluate
sss = StratifiedShuffleSplit(n_splits=25, test_size=0.2, random_state=42)

results = []
for i, (train_idx, test_idx) in enumerate(sss.split(df, df['label'])):
    test_df, y_test, y_pred = meta_fair_classifier_train_and_predict(
        df,
        train_idx,
        test_idx,
        protected,
        privileged_value,
        unprivileged_value,
        favorable,
        unfavorable) 
    
    m = compute_metrics(
        test_df, y_test, y_pred,
        protected, privileged_value, unprivileged_value
    )
    results.append(m)

# 3) Aggregate
compas_race_metrics = pd.DataFrame(results)
compas_race_metrics_agg = compas_race_metrics.agg(['mean','std'])
print(compas_race_metrics_agg)

/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:38: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total


TypeError: cannot unpack non-iterable NoneType object

plot_title = 'Compas Inprocessing Meta Fair Classifier: Baseline - Race'
fig = compare_viz_metrics_2x3(baseline_race_agg, compas_race_metrics_agg, 'Baseline', 'Race', plot_title)
fname    = plot_title.replace(' ', '_').replace('(', '').replace(')', '')
out_path = os.path.join(out_dir_plots, f'{fname}.png')
fig.savefig(out_path)
fig.show()

## default MFC, sex

In [10]:
# 1) Retrieve data
protected           = 'sex'
privileged_value    = 0.0
unprivileged_value  = 1.0

ds, df = load_compas_sex()  
favorable = ds.favorable_label      # COMPAS: 0.0
unfavorable = ds.unfavorable_label  # COMPAS: 1.0
feature_cols = [c for c in df.columns if c not in ('label','sex')]

# 2) Run experiment, Evaluate
sss = StratifiedShuffleSplit(n_splits=25, test_size=0.2, random_state=42)

results = []
for i, (train_idx, test_idx) in enumerate(sss.split(df, df['label'])):
    test_df, y_test, y_pred = meta_fair_classifier_train_and_predict(
    df,
    train_idx,
    test_idx,
    protected,
    privileged_value,
    unprivileged_value,
    favorable,
    unfavorable)
    
    m = compute_metrics(
        test_df, y_test, y_pred,
        protected, privileged_value, unprivileged_value
    )
    results.append(m)

# 3) Aggregate
compas_sex_metrics = pd.DataFrame(results)
compas_sex_metrics_agg = compas_sex_metrics.agg(['mean','std'])
print(compas_race_metrics_agg)

/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:38: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total


TypeError: cannot unpack non-iterable NoneType object

## DEBUG, check what the issue is --------------------------- 

In [11]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.inprocessing import MetaFairClassifier

def debug_celis_meta(train_bld, protected, privileged_value, unprivileged_value):
    labels = train_bld.labels.flatten()
    sens   = train_bld.protected_attributes.flatten()

    # Masks for each cell
    mask_priv      = (sens == privileged_value)
    mask_unpriv    = (sens == unprivileged_value)
    mask_fav       = (labels == train_bld.favorable_label)
    mask_unfav     = (labels == train_bld.unfavorable_label)

    count_P_plus   = int(np.logical_and(mask_priv,    mask_fav).sum())
    count_P_minus  = int(np.logical_and(mask_priv,    mask_unfav).sum())
    count_U_plus   = int(np.logical_and(mask_unpriv,  mask_fav).sum())
    count_U_minus  = int(np.logical_and(mask_unpriv,  mask_unfav).sum())

    print("---- Four‐cell counts in this training fold ----")
    print(f" Privileged ∧ Favorable   = {count_P_plus}")
    print(f" Privileged ∧ Unfavorable = {count_P_minus}")
    print(f" Unprivileged ∧ Favorable = {count_U_plus}")
    print(f" Unprivileged ∧ Unfavorable = {count_U_minus}")
    print()

    # Zero‐variance features (after AIF360’s encoding)
    actual_feature_names = train_bld.feature_names
    df_train_features = pd.DataFrame(train_bld.features, columns=actual_feature_names)
    zero_var_cols = [col for col in actual_feature_names if df_train_features[col].std() == 0]

    if zero_var_cols:
        print("Zero‐variance feature(s) in this fold (after encoding):")
        for col in zero_var_cols:
            print("  -", col)
    else:
        print("No zero‐variance features in this fold (after encoding).")
    print()

    # Perfect separability check: can X → protected_group?
    X = train_bld.features
    z = sens  # 0.0/1.0 for race
    lr = LogisticRegression(solver='liblinear', max_iter=5000)
    lr.fit(X, z)
    train_acc = lr.score(X, z)

    print(f"Logistic regression accuracy (features → protected_group): {train_acc:.3f}")
    if np.isclose(train_acc, 1.0):
        print("  → Perfect (or near‐perfect) separability by protected group detected!")
    else:
        print("  → No perfect separability (accuracy < 1.0).")
    print("-----------------------------------------------------\n")


def meta_fair_classifier_train_and_predict(
    df: pd.DataFrame,
    train_idx: np.ndarray,
    test_idx: np.ndarray,
    protected: str,
    privileged_value: float,
    unprivileged_value: float,
    favorable_label: float,
    unfavorable_label: float
):
    # 1) Split into train/test DataFrames (original, unscaled)
    train_df = df.iloc[train_idx].reset_index(drop=True)
    test_df  = df.iloc[test_idx].reset_index(drop=True)

    # 2) Identify feature columns (exclude 'label' and protected attr)
    feature_cols = [c for c in df.columns if c not in ('label', protected)]

    # 3) Scale numeric features
    scaler     = RobustScaler()
    train_vals = scaler.fit_transform(train_df[feature_cols])
    test_vals  = scaler.transform(test_df[feature_cols])

    # 4) Rebuild DataFrames for AIF360 (protected, label, and scaled features)
    train_scaled = train_df[[protected, 'label']].copy().reset_index(drop=True)
    train_scaled[feature_cols] = train_vals

    test_scaled = test_df[[protected, 'label']].copy().reset_index(drop=True)
    test_scaled[feature_cols] = test_vals

    # 5) Wrap as AIF360 BinaryLabelDataset
    train_bld = BinaryLabelDataset(
        df=train_scaled,
        label_names=['label'],
        protected_attribute_names=[protected],
        favorable_label=favorable_label,
        unfavorable_label=unfavorable_label,
        privileged_protected_attributes=[[privileged_value]],
        unprivileged_protected_attributes=[[unprivileged_value]]
    )
    test_bld = BinaryLabelDataset(
        df=test_scaled,
        label_names=['label'],
        protected_attribute_names=[protected],
        favorable_label=favorable_label,
        unfavorable_label=unfavorable_label,
        privileged_protected_attributes=[[privileged_value]],
        unprivileged_protected_attributes=[[unprivileged_value]]
    )

    # ===== Insert debug prints here =====
    print(f"\n=== DEBUG for fold (train size {len(train_idx)}) ===")
    debug_celis_meta(train_bld, protected, privileged_value, unprivileged_value)
    # =====================================

    # 6) Train MetaFairClassifier with default tau=0.5
    mfc = MetaFairClassifier(
        tau=0.5,
        sensitive_attr=protected,
        type='sr',
        seed=42
    )

    try:
        mfc.fit(train_bld)
    except Exception as e:
        print(f"MetaFairClassifier.fit failed on this fold: {e}")
        # Return test_df and y_test, but y_pred = None so caller knows it failed
        y_test = test_df['label'].values
        return test_df, y_test, None

    # 7) If fit succeeded, predict
    try:
        pred_bld = mfc.predict(test_bld)
        y_pred = pred_bld.labels.ravel()
    except Exception as e:
        print(f"MetaFairClassifier.predict failed on this fold: {e}")
        y_test = test_df['label'].values
        return test_df, y_test, None

    # 8) Return the unscaled test DataFrame + true labels + MetaFair predictions
    y_test = test_df['label'].values
    return test_df, y_test, y_pred


In [ ]:
protected           = 'race'
privileged_value    = 0.0
unprivileged_value  = 1.0

ds, df = load_compas_race()
favorable   = ds.favorable_label    # e.g. 0.0
unfavorable = ds.unfavorable_label  # e.g. 1.0

sss = StratifiedShuffleSplit(n_splits=25, test_size=0.2, random_state=42)

results = []
for fold_i, (train_idx, test_idx) in enumerate(sss.split(df, df['label']), start=1):
    print(f"********** Fold {fold_i} **********")

    test_df, y_test, y_pred = meta_fair_classifier_train_and_predict(
        df,
        train_idx,
        test_idx,
        protected,
        privileged_value,
        unprivileged_value,
        favorable,
        unfavorable
    )

    if y_pred is None:
        # MetaFair failed on this fold: record or skip
        print("Skipping metrics for this fold because MetaFair failed.\n")
        continue


********** Fold 1 **********

=== DEBUG for fold (train size 5771) ===
---- Four‐cell counts in this training fold ----
 Privileged ∧ Favorable   = 1989
 Privileged ∧ Unfavorable = 1814
 Unprivileged ∧ Favorable = 1181
 Unprivileged ∧ Unfavorable = 787

No zero‐variance features in this fold (after encoding).

Logistic regression accuracy (features → protected_group): 1.000
  → Perfect (or near‐perfect) separability by protected group detected!
-----------------------------------------------------



/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:38: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total


MetaFairClassifier.fit failed on this fold: cannot unpack non-iterable NoneType object
Skipping metrics for this fold because MetaFair failed.

********** Fold 2 **********

=== DEBUG for fold (train size 5771) ===
---- Four‐cell counts in this training fold ----
 Privileged ∧ Favorable   = 2002
 Privileged ∧ Unfavorable = 1827
 Unprivileged ∧ Favorable = 1168
 Unprivileged ∧ Unfavorable = 774

No zero‐variance features in this fold (after encoding).

Logistic regression accuracy (features → protected_group): 1.000
  → Perfect (or near‐perfect) separability by protected group detected!
-----------------------------------------------------



/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:38: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total


MetaFairClassifier.fit failed on this fold: cannot unpack non-iterable NoneType object
Skipping metrics for this fold because MetaFair failed.

********** Fold 3 **********

=== DEBUG for fold (train size 5771) ===
---- Four‐cell counts in this training fold ----
 Privileged ∧ Favorable   = 1970
 Privileged ∧ Unfavorable = 1829
 Unprivileged ∧ Favorable = 1200
 Unprivileged ∧ Unfavorable = 772

No zero‐variance features in this fold (after encoding).

Logistic regression accuracy (features → protected_group): 1.000
  → Perfect (or near‐perfect) separability by protected group detected!
-----------------------------------------------------



...

/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:38: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total


MetaFairClassifier.fit failed on this fold: cannot unpack non-iterable NoneType object
Skipping metrics for this fold because MetaFair failed.

********** Fold 24 **********

=== DEBUG for fold (train size 5771) ===
---- Four‐cell counts in this training fold ----
 Privileged ∧ Favorable   = 1982
 Privileged ∧ Unfavorable = 1824
 Unprivileged ∧ Favorable = 1188
 Unprivileged ∧ Unfavorable = 777

No zero‐variance features in this fold (after encoding).

Logistic regression accuracy (features → protected_group): 1.000
  → Perfect (or near‐perfect) separability by protected group detected!
-----------------------------------------------------



/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:38: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total


MetaFairClassifier.fit failed on this fold: cannot unpack non-iterable NoneType object
Skipping metrics for this fold because MetaFair failed.

********** Fold 25 **********

=== DEBUG for fold (train size 5771) ===
---- Four‐cell counts in this training fold ----
 Privileged ∧ Favorable   = 1964
 Privileged ∧ Unfavorable = 1843
 Unprivileged ∧ Favorable = 1206
 Unprivileged ∧ Unfavorable = 758

No zero‐variance features in this fold (after encoding).

Logistic regression accuracy (features → protected_group): 1.000
  → Perfect (or near‐perfect) separability by protected group detected!
-----------------------------------------------------

MetaFairClassifier.fit failed on this fold: cannot unpack non-iterable NoneType object
Skipping metrics for this fold because MetaFair failed.



/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:38: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total


In [14]:
protected           = 'sex'
privileged_value    = 0.0
unprivileged_value  = 1.0

ds, df = load_compas_sex()
favorable   = ds.favorable_label    # e.g. 0.0
unfavorable = ds.unfavorable_label  # e.g. 1.0

sss = StratifiedShuffleSplit(n_splits=25, test_size=0.2, random_state=42)

results = []
for fold_i, (train_idx, test_idx) in enumerate(sss.split(df, df['label']), start=1):
    print(f"********** Fold {fold_i} **********")

    test_df, y_test, y_pred = meta_fair_classifier_train_and_predict(
        df,
        train_idx,
        test_idx,
        protected,
        privileged_value,
        unprivileged_value,
        favorable,
        unfavorable
    )

    if y_pred is None:
        # MetaFair failed on this fold: record or skip
        print("Skipping metrics for this fold because MetaFair failed.\n")
        continue


********** Fold 1 **********

=== DEBUG for fold (train size 5771) ===
---- Four‐cell counts in this training fold ----
 Privileged ∧ Favorable   = 2460
 Privileged ∧ Unfavorable = 2207
 Unprivileged ∧ Favorable = 710
 Unprivileged ∧ Unfavorable = 394

No zero‐variance features in this fold (after encoding).

Logistic regression accuracy (features → protected_group): 1.000
  → Perfect (or near‐perfect) separability by protected group detected!
-----------------------------------------------------



/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:38: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + 

MetaFairClassifier.fit failed on this fold: cannot unpack non-iterable NoneType object
Skipping metrics for this fold because MetaFair failed.

********** Fold 2 **********

=== DEBUG for fold (train size 5771) ===
---- Four‐cell counts in this training fold ----
 Privileged ∧ Favorable   = 2449
 Privileged ∧ Unfavorable = 2189
 Unprivileged ∧ Favorable = 721
 Unprivileged ∧ Unfavorable = 412

No zero‐variance features in this fold (after encoding).

Logistic regression accuracy (features → protected_group): 1.000
  → Perfect (or near‐perfect) separability by protected group detected!
-----------------------------------------------------

MetaFairClassifier.fit failed on this fold: cannot unpack non-iterable NoneType object
Skipping metrics for this fold because MetaFair failed.

********** Fold 3 **********

=== DEBUG for fold (train size 5771) ===
---- Four‐cell counts in this training fold ----
 Privileged ∧ Favorable   = 2440
 Privileged ∧ Unfavorable = 2200
 Unprivileged ∧ Favorab

/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:38: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total


MetaFairClassifier.fit failed on this fold: cannot unpack non-iterable NoneType object
Skipping metrics for this fold because MetaFair failed.

********** Fold 4 **********

=== DEBUG for fold (train size 5771) ===
---- Four‐cell counts in this training fold ----
 Privileged ∧ Favorable   = 2469
 Privileged ∧ Unfavorable = 2188
 Unprivileged ∧ Favorable = 701
 Unprivileged ∧ Unfavorable = 413

No zero‐variance features in this fold (after encoding).

Logistic regression accuracy (features → protected_group): 1.000
  → Perfect (or near‐perfect) separability by protected group detected!
-----------------------------------------------------



...

MetaFairClassifier.fit failed on this fold: cannot unpack non-iterable NoneType object
Skipping metrics for this fold because MetaFair failed.

********** Fold 24 **********

=== DEBUG for fold (train size 5771) ===
---- Four‐cell counts in this training fold ----
 Privileged ∧ Favorable   = 2444
 Privileged ∧ Unfavorable = 2206
 Unprivileged ∧ Favorable = 726
 Unprivileged ∧ Unfavorable = 395

No zero‐variance features in this fold (after encoding).

Logistic regression accuracy (features → protected_group): 1.000
  → Perfect (or near‐perfect) separability by protected group detected!
-----------------------------------------------------



/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:38: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total


MetaFairClassifier.fit failed on this fold: cannot unpack non-iterable NoneType object
Skipping metrics for this fold because MetaFair failed.

********** Fold 25 **********

=== DEBUG for fold (train size 5771) ===
---- Four‐cell counts in this training fold ----
 Privileged ∧ Favorable   = 2446
 Privileged ∧ Unfavorable = 2195
 Unprivileged ∧ Favorable = 724
 Unprivileged ∧ Unfavorable = 406

No zero‐variance features in this fold (after encoding).

Logistic regression accuracy (features → protected_group): 1.000
  → Perfect (or near‐perfect) separability by protected group detected!
-----------------------------------------------------



/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:36: RuntimeWarning: invalid value encountered in divide
  prob_y_1 = (prob_1_1 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:37: RuntimeWarning: invalid value encountered in divide
  prob_z_0 = (prob_m1_0 + prob_1_0) / total
/opt/anaconda3/lib/python3.11/site-packages/aif360/algorithms/inprocessing/celisMeta/StatisticalRate.py:38: RuntimeWarning: invalid value encountered in divide
  prob_z_1 = (prob_m1_1 + prob_1_1) / total


MetaFairClassifier.fit failed on this fold: cannot unpack non-iterable NoneType object
Skipping metrics for this fold because MetaFair failed.



# --------------

# 1) Save the aggregated metrics 
save_agg_metrics(
    dataset_name      = dataset_name,
    mitigation_name   = mitigation_name,
    race_agg_df       = compas_race_metrics_agg,
    sex_agg_df        = compas_sex_metrics_agg,
    pipeline_stage    = pipeline_stage   
)

# 2) Save the raw metrics
save_raw_metrics(
    dataset_name      = dataset_name,
    mitigation_name   = mitigation_name,
    race_raw_df       = compas_race_metrics,
    sex_raw_df        = compas_sex_metrics,
    pipeline_stage    = pipeline_stage
)
